# Advanced Plotting

This provides some additional examples of more advanced sea ice fields. Here we introduce the concept of the subgridscale ice thickness distribution (ITD). This means we have a fraction of ice in each grid cell that is binned into thickness categories.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpath
from matplotlib.gridspec import GridSpec
import pop_tools
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os

For these exercises we will need to import multiple variables, below is an example of one way to do so.

In [ ]:
monthly_output_path = "/glade/campaign/cgd/cesm/CESM2-LE/ice/proc/tseries/month_1"
run_name = "b.e21.BHISTcmip6.f09_g17.LE2-1001.001"

var_names = ['aice',
             'aicen',
             'vsnon',
             'hs',
             'fsens',
             'fsens_ai',
            ]

da_list = []

for var_name in var_names:
    files = os.path.join(monthly_output_path, var_name,
                         run_name + ".cice.h." + var_name + ".*")
    ds_in = xr.open_mfdataset(files)
    da_list.append(ds_in[var_name])
    del ds_in

ds = xr.merge(da_list)

del da_list

The next step is to read in some grid information for the `gx1v7` dipole grid used in POP and CICE. We will read in three main variables: `tarea`, `TLAT`, and `TLON`. These are the areas of the gridcells along with the latitudes and longitudes of the gridcell centers. Also, we will print the latitude array `TLAT` to see the metadata.

In [ ]:
# get pop grid grid cell areas
grid = pop_tools.get_grid('POP_gx1v7')

# convert tarea to m^2
with xr.set_options(keep_attrs=True):
    grid['TAREA'] = grid['TAREA']/(1e4)
grid['TAREA'].attrs['units'] = 'm^2'

We will merge in three main variables: `tarea`, `TLAT`, and `TLON`. These are the areas of the gridcells along with the latitudes and longitudes of the gridcell centers. Note that this overwrites the dataset object from above.

In [ ]:
ds = xr.merge([ds.drop(['TLAT', 'TLON', 'ULAT', 'ULON']),
               grid[['TLAT', 'TLONG', 'TAREA']].rename_dims({'nlat':'nj','nlon':'ni'})],
              compat='identical', combine_attrs='no_conflicts')

In [ ]:
ds

## Example 1: Plot per-category ice area

Compare the dataset in this notebook with `aice` in the basics notebook. Notice that in this case we have an additional category dimension `nc`. `aicen` is the per-category ice area fraction. We demonstrate plotting a per-category variable below. We also plot the full sea ice concentration in the final plot.

In [ ]:
# make circular boundary for polar stereographic circular plots
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

cmap = plt.cm.get_cmap('Blues_r')  

# create figure with subplots
fig, axs = plt.subplots(3, 2, figsize=(20,30),
                       subplot_kw={'projection':ccrs.NorthPolarStereo()})
axs = np.ravel(axs)

# this creates a subplot for each ITD category
for i in ds.nc.values:
    ax = axs[i]
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
    ax.set_extent([0.005, 360, 90, 55], crs=ccrs.PlateCarree())
    this=ax.pcolormesh(ds['TLONG'],
                       ds['TLAT'],
                       ds['aicen'].sel({'time':'1850-02-01 00:00:00',
                                        'nc':i}).squeeze(),
                       cmap=cmap,vmax=1,vmin=0,
                       transform=ccrs.PlateCarree())
    plt.colorbar(this,orientation='vertical',fraction=0.04,pad=0.01)
    ax.set_title('Area Fraction Category ' + str(i+1))

# gridcell mean aice in the final subplot
ax = axs[-1]
ax.set_boundary(circle, transform=ax.transAxes)
ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
ax.set_extent([0.005, 360, 90, 55], crs=ccrs.PlateCarree())
this=ax.pcolormesh(ds['TLONG'],
                   ds['TLAT'],
                   ds['aice'].sel({'time':'1850-02-01 00:00:00'}).squeeze(),
                   cmap=cmap,vmax=1,vmin=0,
                   transform=ccrs.PlateCarree())
plt.colorbar(this,orientation='vertical',fraction=0.04,pad=0.01)
ax.set_title('Sea Ice Concentration')

## Example 2: Plot per-category snow thickness

Internally, the model actually stores the snow **volume** for each category, not the thickness. To get the thickness we need to divide `vsnon` by `aicen` (the per category area.

We will also plot the grid cell mean snow thickness. How does this compare with the per-category values?

In [ ]:
ds['hsn'] = ds['vsnon'] / ds['aicen']

In [ ]:
# Max snow depth for colorbars
hs_max = 0.5 

# make circular boundary for polar stereographic circular plots
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

cmap = plt.cm.get_cmap('Blues_r')  


fig, axs = plt.subplots(3, 2, figsize=(20,30),
                       subplot_kw={'projection':ccrs.NorthPolarStereo()})
axs = np.ravel(axs)

for i in ds.nc.values:
    ax = axs[i]
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
    ax.set_extent([0.005, 360, 90, 55], crs=ccrs.PlateCarree())
    this=ax.pcolormesh(ds['TLONG'],
                       ds['TLAT'],
                       ds['hsn'].sel({'time':'1850-02-01 00:00:00',
                                        'nc':i}).squeeze(),
                       cmap=cmap,vmax=hs_max,vmin=0,
                       transform=ccrs.PlateCarree())
    plt.colorbar(this,orientation='vertical',fraction=0.04,pad=0.01)
    ax.set_title('Snow Depth (m) Category ' + str(i+1))

# gridcell mean snow volume in the final subplot
ax = axs[-1]
ax.set_boundary(circle, transform=ax.transAxes)
ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
ax.set_extent([0.005, 360, 90, 55], crs=ccrs.PlateCarree())
this=ax.pcolormesh(ds['TLONG'],
                   ds['TLAT'],
                   ds['hs'].sel({'time':'1850-02-01 00:00:00'}).squeeze(),
                   cmap=cmap,vmax=hs_max,vmin=0,
                   transform=ccrs.PlateCarree())
plt.colorbar(this,orientation='vertical',fraction=0.04,pad=0.01)
ax.set_title('Average Snow Depth (m)')

## Example 3: Ice area related tracer

The default CICE outputs are averaged over the entire grid cell, including the open water. Thus if a grid cell happened to be half covered in 1-m-thick ice and half open water then `hi` would be 0.5 m. Some tracers are written out just for the ice-covered area of the grid cell. These are indicated by have `_ai` appended to the variable name.

Below is an example of this for the sensible heat flux.

In [ ]:
ds['fsens_diff'] = ds['fsens_ai'] - ds['fsens']

In [ ]:
# Min and max
mins = [-60, -60, -30, 0]
maxs = [60, 60, 30, 1]

# make circular boundary for polar stereographic circular plots
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)

cmap = plt.cm.get_cmap('RdBu')  

vars_to_plt = ['fsens_ai', 'fsens', 'fsens_diff', 'aice']

fig, axs = plt.subplots(2, 2, figsize=(20,20),
                       subplot_kw={'projection':ccrs.NorthPolarStereo()})
axs = np.ravel(axs)

# set up the plots for sensible heat flux over the sea ice, gridcell mean sensible heat flux,
# difference, and the gridcell mean ice concentration aice

for i in np.arange(4):
    if i == 3:
        cmap = plt.cm.get_cmap('Blues_r')
    ax = axs[i]
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.add_feature(cfeature.LAND,zorder=100,edgecolor='k')
    ax.set_extent([0.005, 360, 90, 55], crs=ccrs.PlateCarree())
    this=ax.pcolormesh(ds['TLONG'],
                       ds['TLAT'],
                       ds[vars_to_plt[i]].sel({'time':'1850-02-01 00:00:00'}
                                             ).squeeze(),
                       cmap=cmap,vmax=maxs[i],vmin=mins[i],
                       transform=ccrs.PlateCarree())
    plt.colorbar(this,orientation='vertical',fraction=0.04,pad=0.01)
    ax.set_title(vars_to_plt[i])



Where are the differences most pronounced and why?